In [1]:
%%time 

from IPython.display import display_html, clear_output, Markdown;
from gc import collect;

from copy import deepcopy;
import pandas as pd;
import numpy as np;
import joblib;
from os import system, getpid, walk;
from psutil import Process;
import ctypes;

from pprint import pprint;
from colorama import Fore, Style, init;
from warnings import filterwarnings;
filterwarnings('ignore');

from tqdm.notebook import tqdm;

print();
collect();


CPU times: total: 562 ms
Wall time: 1.05 s


In [2]:
import lightgbm
print('lightgbm版本：',lightgbm.__version__)

lightgbm版本： 3.3.2


In [9]:
%%time 

from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
                                     StratifiedKFold as SKF,
                                     KFold, 
                                     RepeatedKFold as RKF, 
                                     cross_val_score);

from lightgbm import log_evaluation, early_stopping, LGBMRegressor as LGBMR;
from xgboost import XGBRegressor as XGBR;
from catboost import CatBoostRegressor as CBR;
from sklearn.ensemble import HistGradientBoostingRegressor as HGBR;
from sklearn.metrics import mean_squared_error as mse, make_scorer;

print();
collect();


CPU times: total: 62.5 ms
Wall time: 60.5 ms


In [10]:
%%time
  
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL); 
    
def GetMemUsage():
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM memory GB usage = {memory_use :.4}";

from sklearn import set_config; 
set_config(transform_output = "pandas");
pd.set_option('display.max_columns', 50);
pd.set_option('display.max_rows', 50);

print();
collect();


CPU times: total: 62.5 ms
Wall time: 64.5 ms


In [28]:
%%time 

class CFG:


    version_nb         = 1; # 版本号 没做一个调整 手动+1 

    gpu_switch         = "ON"; # 是否需要GPU
    state              = 42; # 随机种子 random seed
    mdl_path           = f''; # 模型路径 使用 joblib 转存训练后的模型
    methods            = ["CBR", "LGBMR", "HGBR"]; # 所有训练过的模型方法 基于推理的内存约束选择一个以上 所有的训练后方法都需要存在
    n_splits           = 5; # 交叉检验 分割数
    n_repeats          = 1; # 交叉检验 重复数
    nbrnd_erly_stp     = 100 ; # 提前停止轮次
    mdlcv_mthd         = 'KF'; # 模型交叉验证选择

print();
PrintColor(f"--> Configuration done!\n");
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);


--> Configuration done!


RAM memory GB usage = 8.23
CPU times: total: 93.8 ms
Wall time: 89.8 ms


In [29]:
%%time 

all_cv= {'KF'  : KFold(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state),
         'RKF' : RKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'RSKF': RSKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'SKF' : SKF(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state)
        };

def ScoreMetric(ytrue, ypred)-> float:
    return mse(ytrue, ypred);

myscorer = make_scorer(ScoreMetric, greater_is_better = False, needs_proba=False,);

print();
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 8.23
CPU times: total: 93.8 ms
Wall time: 90.8 ms


In [30]:
%%time 


Mdl_Master = \
{'CBR': CBR(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
               'objective'           : "RMSE",
               'eval_metric'         : "RMSE",
               'bagging_temperature' : 0.5,
#                'colsample_bylevel'   : 0.7, # rsm on GPU is supported for pairwise modes only gpu训练需注释
               'iterations'          : 500,
               'learning_rate'       : 0.065,
               'od_wait'             : 25,
               'max_depth'           : 7,
               'l2_leaf_reg'         : 1.5,
               'min_data_in_leaf'    : 1000,
               'random_strength'     : 0.65, 
               'verbose'             : 0,
               'use_best_model'      : True,
              }
           ), 

  'LGBMR': LGBMR(**{'device'            : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                    'objective'         : 'regression_l1',
                    'boosting_type'     : 'gbdt',
                    'random_state'      : CFG.state,
                    'colsample_bytree'  : 0.7,
                    'subsample'         : 0.65,
                    'learning_rate'     : 0.065,
                    'max_depth'         : 6,
                    'n_estimators'      : 500,
                    'num_leaves'        : 150,  
                    'reg_alpha'         : 0.01,
                    'reg_lambda'        : 3.25,
                    'verbose'           : -1,
                   }
                ),

  'XGBR': XGBR(**{'tree_method'        : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                  'objective'          : 'reg:absoluteerror',
                  'random_state'       : CFG.state,
                  'colsample_bytree'   : 0.7,
                  'learning_rate'      : 0.07,
                  'max_depth'          : 6,
                  'n_estimators'       : 500,                         
                  'reg_alpha'          : 0.025,
                  'reg_lambda'         : 1.75,
                  'min_child_weight'   : 1000,
                  'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                 }
              ),

  "HGBR" : HGBR(loss              = 'squared_error',
                learning_rate     = 0.075,
                early_stopping    = True,
                max_iter          = 200,
                max_depth         = 6,
                min_samples_leaf  = 1500,
                l2_regularization = 1.75,
                scoring           = myscorer,
                random_state      = CFG.state,
               )
};

print();
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 8.222
CPU times: total: 78.1 ms
Wall time: 85.3 ms


In [31]:
X = pd.read_parquet(f"XTrIntCmpNewFtre.parquet");
y = pd.read_parquet(f"Ytrain.parquet").squeeze();  
PrintColor(f"---> Train shapes for code testing = {X.shape} {y.shape}");

---> Train shapes for code testing = (5237892, 52) (5237892,)


In [32]:
%%time 

methods = CFG.methods;
system('mkdir BaselineML');
model_path = CFG.mdl_path;
cv = all_cv[CFG.mdlcv_mthd];
Scores = pd.DataFrame(index = range(CFG.n_splits * CFG.n_repeats),
                      columns = methods).fillna(0).astype(np.float32);

FtreImp = pd.DataFrame(index = X.columns, columns = [methods]).fillna(0);

print();
collect();
# libc.malloc_trim(0);

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 8.241
CPU times: total: 93.8 ms
Wall time: 104 ms


In [33]:
%%time 

PrintColor(f"\n{'=' * 25} ML Training {'=' * 25}\n");
# 初始化 交叉验证分割
# CFG mdlcv_mthd 交叉训练验证选择
# Initializing CV splitting:-       
for fold_nb, (train_idx, dev_idx) in tqdm(enumerate(cv.split(X, y)), 
                                          f"{CFG.mdlcv_mthd} CV {CFG.n_splits}x{CFG.n_repeats}"
                                         ): 
    # Creating the cv folds:-    
    Xtr  = X.iloc[train_idx];   
    Xdev = X.iloc[dev_idx];
    ytr  = y.iloc[train_idx];
    ydev = y.iloc[dev_idx];

    PrintColor(f"-------> Fold{fold_nb} <-------");
    # Fitting the models:- 
    for method in methods:
        model = Mdl_Master[method];
        if method == "LGBMR":
            model.fit(Xtr, ytr, 
                      eval_set = [(Xdev, ydev)], 
                      verbose = 0,
                      eval_metric = "mse",
                      callbacks = [log_evaluation(0,), 
                                   early_stopping(CFG.nbrnd_erly_stp, verbose = False)], 
                     );

        elif method == "XGBR":
            model.fit(Xtr, ytr, 
                      eval_set = [(Xdev, ydev)], 
                      verbose = 0, 
                      eval_metric = "mse",
                     );  

        elif method == "CBR":
            model.fit(Xtr, ytr, 
                      eval_set = [(Xdev, ydev)], 
                      verbose = 0, 
                      early_stopping_rounds = CFG.nbrnd_erly_stp,
                     ); 

        else:
            model.fit(Xtr, ytr);

        #  Saving the model for later usage:-
        joblib.dump(model, CFG.mdl_path + f'{method}V{CFG.version_nb}Fold{fold_nb}.model');

        # Creating OOF scores:-
        score = ScoreMetric(ydev, model.predict(Xdev));
        Scores.at[fold_nb, method] = score;
        num_space = 6- len(method);
        PrintColor(f"---> {method} {' '* num_space} OOF = {score:.5f}", 
                   color = Fore.MAGENTA);  
        del num_space, score;

        # Collecting feature importances:-
        try:
            FtreImp[method] = \
            FtreImp[method].values + (model.feature_importances_ / (CFG.n_splits * CFG.n_repeats));
        except:
            pass;

        collect();

    PrintColor(GetMemUsage());
    print();
    del Xtr, ytr, Xdev, ydev;
    collect();

# clear_output();
PrintColor(f"\n---> OOF scores across methods <---\n");
Scores.index.name = "FoldNb";
Scores.index = Scores.index + 1;
display(Scores.style.format(precision = 5).\
        background_gradient(cmap = "Pastel1")
       );

PrintColor(f"\n---> Mean OOF scores across methods <---\n");
display(Scores.mean());

try: FtreImp.to_csv(CFG.mdl_path + f"FtreImp_V{CFG.version_nb}.csv");
except: pass;
        
collect();
print();
# libc.malloc_trim(0);

PrintColor(f"\n" + GetMemUsage(), color = Fore.GREEN);


========================= ML Training =========================



KF CV 5x1: 0it [00:00, ?it/s]

-------> Fold0 <-------
---> CBR     OOF = 84.29983
---> LGBMR   OOF = 84.85627
---> HGBR    OOF = 85.26530
RAM memory GB usage = 8.346

-------> Fold1 <-------
---> CBR     OOF = 84.49962
---> LGBMR   OOF = 84.95976
---> HGBR    OOF = 85.35805
RAM memory GB usage = 8.355

-------> Fold2 <-------
---> CBR     OOF = 84.94060
---> LGBMR   OOF = 85.42588
---> HGBR    OOF = 85.87322
RAM memory GB usage = 8.33

-------> Fold3 <-------
---> CBR     OOF = 84.77606
---> LGBMR   OOF = 85.24124
---> HGBR    OOF = 85.65652
RAM memory GB usage = 8.351

-------> Fold4 <-------
---> CBR     OOF = 84.34216
---> LGBMR   OOF = 84.90664
---> HGBR    OOF = 85.33493
RAM memory GB usage = 8.337


---> OOF scores across methods <---



,CBR,LGBMR,HGBR
FoldNb,,,
1,84.29983,84.85627,85.26530
2,84.49962,84.95976,85.35805
3,84.94060,85.42588,85.87322
4,84.77606,85.24124,85.65652
5,84.34216,84.90664,85.33493



---> Mean OOF scores across methods <---



CBR      84.571653
LGBMR    85.077961
HGBR     85.497602
dtype: float64



RAM memory GB usage = 6.645
CPU times: total: 4h 25min 19s
Wall time: 22min 31s


In [27]:
# mae 和 rsme